<a href="https://colab.research.google.com/github/drsimonturega/ir_spec_man/blob/main/asptocsv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Infrared spectral manipulation: *.asp to *.csv
### One spectra at a time
### You need to be logged into your google acount to use this code
### There are three sections of code that need to be run in sequence;
- ### Lab book code for this experiemnt
- ### Upload your .asp spectra file (.asp from a Agilent ATR-FTIR spectrometer)
- ### Extract data from *.asp to transposed *.csv

### This will provide a *.zip file for down load your orginal file and your spectra as a transposed *.csv.

In [18]:
# @title ## Lab book code for this experiemnt
# @markdown ### Enter the lab book code for this experiemnt
lab_book_code = '03072024_4' # @param {type:"string"}
import os
import shutil

os.chdir("/content")
try:
    os.mkdir(lab_book_code)
except FileExistsError:
    print(f'A folder for {lab_book_code} already exists you may over write your\
        data if you continue')
    #the lines below remove the folder! not currently implimented
    #os.chdir("..")
    #shutil.rmtree(lab_book_code)
try:
    !pwd
    os.chdir(lab_book_code)
    !pwd
except FileNotFoundError:
    print(f'try again')



/content
/content/03072024_3


## Upload your .asp spectra file (.asp from a Agilent ATR-FTIR spectrometer)



In [19]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
asp_input = list(uploaded.keys())
asp_input = str(asp_input[0])

asp_input = asp_input[:-4]
print(f'{asp_input}')

Saving BAS_25_001_2023-06-14T09-55-56.asp to BAS_25_001_2023-06-14T09-55-56.asp
Saving BAS_100_001_2023-06-14T09-58-46.asp to BAS_100_001_2023-06-14T09-58-46.asp
Saving BAS_Buffer_001_2023-06-13T15-54-39.asp to BAS_Buffer_001_2023-06-13T15-54-39.asp
Saving BAS_Buffer_001_2023-06-13T15-56-35.asp to BAS_Buffer_001_2023-06-13T15-56-35.asp
Saving BAS_Water_2023-06-13T15-51-57.asp to BAS_Water_2023-06-13T15-51-57.asp
User uploaded file "BAS_25_001_2023-06-14T09-55-56.asp" with length 17340 bytes
User uploaded file "BAS_100_001_2023-06-14T09-58-46.asp" with length 17358 bytes
User uploaded file "BAS_Buffer_001_2023-06-13T15-54-39.asp" with length 17357 bytes
User uploaded file "BAS_Buffer_001_2023-06-13T15-56-35.asp" with length 17340 bytes
User uploaded file "BAS_Water_2023-06-13T15-51-57.asp" with length 17342 bytes
BAS_25_001_2023-06-14T09-55-56


### Extract data from *.asp to transposed *.csv

In [17]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from sklearn.linear_model import LinearRegression
import statistics
from tabulate import tabulate
#from google.colab import files
import shutil


class asptospectocsv():


    def __init__(self, asp_input):
        return

    def spec_import(self, spec_sht_name):
        #importing spectra from .asp from a waters? ATR-FTIR spectrometer
        spec_inp= pd.DataFrame()
        try:
            spec_inp = pd.read_csv(spec_sht_name + '.asp')
        except PermissionError:
            print('The file we are trying to open may be open in another \
                program, please close and try again!')
        print(spec_inp.head())
        return spec_inp

    def asptodfd(self, spec_inp):
      from numpy import log as ln

      ir_spec_raw = spec_inp

      # we need th col name as a float?
      col_head = list(ir_spec_raw.columns)
      num_spec_points = col_head[(0)]

      # build the x axis (wavenumbers) from the file format
      high_wavnum = ir_spec_raw.iloc[0,0] #highest wavenumber
      high_wavnum = float(high_wavnum)
      low_wavnum = ir_spec_raw.iloc[1,0] #lowest wavenumber
      low_wavnum = float(low_wavnum)
      num_spec_points = float(num_spec_points)
      spec_gap = ( high_wavnum - low_wavnum ) / num_spec_points #calaculate spectral gap
      wav_num = []
      wav_nums = np.arange(low_wavnum, high_wavnum, spec_gap)
      wav_num_rev=[]

      for i in reversed(np.arange(low_wavnum, high_wavnum, spec_gap)): #build the list of wave numbers
        wav_num_rev.append(i)
      #set first col in dataframe as x axis (wavenumber)
      spec_sin = pd.DataFrame()
      spec_sin['wave number / cm^-1'] = wav_num_rev

      #add y axis (transmitance) to dataframe
      spec_trans=[] # a list to transport transmitance data
      spec_trans = ir_spec_raw.iloc[5:,0] #it gets sliced
      spec_trans = spec_trans.reset_index(drop=True) #it gets reindexed
      spec_sin['spec1'] = spec_trans #spec name
      bg = pd.DataFrame.transpose(spec_sin)

      bg.rename(columns=bg.iloc[0,0:], inplace=True) # set row 0 as col titles
      bg = bg.reset_index(drop=True)
      bg = pd.DataFrame.drop(bg, axis=0, index=0)
      print(bg)
      spec_sin = bg
      return spec_sin


    def savendl(self,spec_sin, asp_input):
      print(f'what is this {spec_sin}')
      try: #output results
            spec_sin.to_csv("ir_" + asp_input + ".csv",index=False)
      except PermissionError:
            print('The file we are trying to open may be open in another \
                program, please close and try again!')
      os.chdir("/content")
      shutil.make_archive(lab_book_code, 'zip', lab_book_code)
      files.download(f'{lab_book_code}.zip')
      return

specasp = asptospectocsv(asp_input)
spec_csv = asptospectocsv.spec_import(specasp, asp_input)
final_csv = asptospectocsv.asptodfd(specasp, spec_csv)
asptospectocsv.savendl(specasp, final_csv, asp_input)

           967
0  3999.433499
1   398.825149
2     1.000000
3   128.000000
4     8.000000
   3995.710016  3991.986533  3988.263049  3984.539566  3980.816083  \
1    99.648306    99.539681    99.469755    99.494827    99.627842   

   3977.092600  3973.369116  3969.645633  3965.922150  3962.198666  ...  \
1    99.610026    99.683539    99.725783    99.546103    99.466969  ...   

   432.336498   428.613015   424.889532   421.166048   417.442565   \
1    30.636227    30.474561    30.239649    31.839339    33.023719   

   413.719082   409.995598   406.272115   402.548632   398.825149   
1     33.74774    32.718461    33.983868    35.088657    36.219493  

[1 rows x 967 columns]
what is this    3995.710016  3991.986533  3988.263049  3984.539566  3980.816083  \
1    99.648306    99.539681    99.469755    99.494827    99.627842   

   3977.092600  3973.369116  3969.645633  3965.922150  3962.198666  ...  \
1    99.610026    99.683539    99.725783    99.546103    99.466969  ...   

   432.336

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>